In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# ==========================================
# 1. Load Data from CSV
# ==========================================
# Reads the file 'data.csv' as requested
try:
    df = pd.read_csv('/content/data.csv')
    print("File loaded successfully.")
except FileNotFoundError:
    print("Error: 'data.csv' not found. Please make sure the file is in the same directory.")
    exit()

# Extract Input Features (bedrooms, sqft_living) and Target (price)
# Based on the column names visible in your image
X = df[['bedrooms', 'sqft_living']].values
y = df[['price']].values

# ==========================================
# 2. Data Preprocessing (Crucial)
# ==========================================
# Neural networks require scaled data (e.g., between -1 and 1) to work correctly.
# 'sqft_living' (e.g., 2000) and 'price' (e.g., 300000) are too large for raw training.
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Convert to PyTorch Tensors (Float32 is standard for PyTorch)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

# ==========================================
# 3. Define Neural Network Architecture
# ==========================================
class RegressionNN(nn.Module):
    def __init__(self):
        super(RegressionNN, self).__init__()

        # Hidden Layer: Matches the diagram with 2 input nodes -> 2 hidden nodes
        # This handles weights w11, w12, w21, w22 and biases b1
        self.hidden = nn.Linear(in_features=2, out_features=2)

        # Output Layer: Matches the diagram with 2 hidden nodes -> 1 output node
        # This handles weights v1, v2 and bias b2
        self.output = nn.Linear(in_features=2, out_features=1)

        # Activation Function (Relu or Sigmoid)
        self.activation = nn.ReLU()

    def forward(self, x):
        # Pass through hidden layer (Input -> Hidden)
        x = self.hidden(x)
        x = self.activation(x)

        # Pass through output layer (Hidden -> Output)
        # No activation is used at the output for Regression problems (to allow continuous values)
        x = self.output(x)
        return x

# Initialize the Model
model = RegressionNN()

# ==========================================
# 4. Training Configuration
# ==========================================
criterion = nn.MSELoss()  # Mean Squared Error (Standard for Regression)
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

# ==========================================
# 5. Training Loop
# ==========================================
epochs = 500
print(f"Starting training for {epochs} epochs...")

for epoch in range(epochs):
    # 1. Forward Pass
    predictions = model(X_tensor)
    loss = criterion(predictions, y_tensor)

    # 2. Backward Pass & Optimization
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()        # Calculate gradients
    optimizer.step()       # Update weights

    # Print loss every 50 epochs
    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

print("Training Complete.")

# ==========================================
# 6. (Optional) Test with a sample prediction
# ==========================================
# Let's predict price for a house with 3 bedrooms and 2000 sqft_living
with torch.no_grad():
    sample_input = [[3, 2000]]
    sample_scaled = scaler_x.transform(sample_input)
    sample_tensor = torch.tensor(sample_scaled, dtype=torch.float32)

    predicted_scaled = model(sample_tensor)
    predicted_price = scaler_y.inverse_transform(predicted_scaled.numpy())

    print(f"\nPrediction for {sample_input[0]}: ${predicted_price[0][0]:.2f}")

File loaded successfully.
Starting training for 500 epochs...
Epoch [50/500], Loss: 0.8695
Epoch [100/500], Loss: 0.8434
Epoch [150/500], Loss: 0.8253
Epoch [200/500], Loss: 0.8093
Epoch [250/500], Loss: 0.7941
Epoch [300/500], Loss: 0.7785
Epoch [350/500], Loss: 0.7614
Epoch [400/500], Loss: 0.7434
Epoch [450/500], Loss: 0.7227
Epoch [500/500], Loss: 0.7025
Training Complete.

Prediction for [3, 2000]: $521550.97


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

# ==========================================
# 1. Load Data
# ==========================================
try:
    df = pd.read_csv('/content/data.csv')
except FileNotFoundError:
    print("Error: 'data.csv' not found.")
    exit()

X = df[['bedrooms', 'sqft_living']].values
y = df[['price']].values

# ==========================================
# 2. Preprocessing
# ==========================================
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

# ==========================================
# 3. Define Neural Network
# ==========================================
class RegressionNN(nn.Module):
    def __init__(self):
        super(RegressionNN, self).__init__()
        # 2 Inputs -> 2 Hidden
        self.hidden = nn.Linear(2, 2)
        # 2 Hidden -> 1 Output
        self.output = nn.Linear(2, 1)

        # CHANGED: Sigmoid is often better for very small "toy" networks than ReLU
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.hidden(x)
        x = self.activation(x)
        x = self.output(x)
        return x

model = RegressionNN()

# ==========================================
# 4. Training Config (UPDATED)
# ==========================================
criterion = nn.MSELoss()

# CHANGED: Increased lr to 0.1 and added momentum=0.9 for faster convergence
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# ==========================================
# 5. Training Loop (UPDATED)
# ==========================================
# CHANGED: Increased epochs to 2000 because 500 was not enough
epochs = 2000
print(f"Starting training for {epochs} epochs...")

for epoch in range(epochs):
    predictions = model(X_tensor)
    loss = criterion(predictions, y_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print every 200 epochs to reduce clutter
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

print("Training Complete.")

# ==========================================
# 6. Test Prediction
# ==========================================
with torch.no_grad():
    sample_input = [[3, 2000]]
    sample_scaled = scaler_x.transform(sample_input)
    sample_tensor = torch.tensor(sample_scaled, dtype=torch.float32)

    predicted_scaled = model(sample_tensor)
    predicted_price = scaler_y.inverse_transform(predicted_scaled.numpy())

    print(f"\nInput: 3 Bedrooms, 2000 sqft")
    print(f"Predicted Price: ${predicted_price[0][0]:.2f}")

Starting training for 2000 epochs...
Epoch [200/2000], Loss: 0.4510
Epoch [400/2000], Loss: 0.4464
Epoch [600/2000], Loss: 0.4458
Epoch [800/2000], Loss: 0.4456
Epoch [1000/2000], Loss: 0.4455
Epoch [1200/2000], Loss: 0.4453
Epoch [1400/2000], Loss: 0.4447
Epoch [1600/2000], Loss: 0.4401
Epoch [1800/2000], Loss: 0.4385
Epoch [2000/2000], Loss: 0.4381
Training Complete.

Input: 3 Bedrooms, 2000 sqft
Predicted Price: $496958.97
